### Model 1.5: account for different traffic and pricing depending on time of day

In [1]:
from amplpy import AMPL
import pandas as pd

In [2]:
rideshares = pd.read_csv('./rideshare_tg22.csv')
airports = ['LAX', 'ONT', 'SNA', 'BUR']
car_types = ['UberX', 'UberXL']

In [3]:
ez_rideshares = rideshares#[rideshares['Departure Date'] == '5/11/24']
ez_rideshares.rename(columns={'Departure Time (Pacific Daylight Time)': 'Flight depart'}, inplace=True)

In [4]:
cost_data = {
    ('LAX', 'UberX'): 90, ('LAX', 'UberXL'): 120,
    ('ONT', 'UberX'): 30, ('ONT', 'UberXL'): 40,
    ('SNA', 'UberX'): 90, ('SNA', 'UberXL'): 120,
    ('BUR', 'UberX'): 90, ('BUR', 'UberXL'): 120
}

In [5]:
capacity_data = {
    'UberX': 4,
    'UberXL': 6
}

In [6]:
travel_time_data = {
    'LAX': 1.5,
    'ONT': 0.5,
    'SNA': 1.5,
    'BUR': 1.5
}

In [7]:
pricing_multiplier = [1.00, 1.02, 1.02, 1.03, 1.03, 0.99, 1.04, 1.03, 1.03, 1.08, 1.08, 1.14, 1.14, 1.19, 1.19, 1.20, 1.20, 1.30, 1.14, 1.10, 1.13, 1.12, 1.15, 1.17, 1.16, 1.17, 1.17, 1.20, 1.20, 1.19, 1.23, 1.22, 1.26, 1.24, 1.27, 1.25, 1.25, 1.18, 1.18, 1.19, 1.19, 1.17, 1.08, 1.12, 1.12, 1.07, 1.07, 1.00 ]

In [8]:
ez_rideshares['Flight depart'] = pd.to_datetime(ez_rideshares['Flight depart'], format='%H:%M')
ez_rideshares['Flight times'] = ez_rideshares['Flight depart'].dt.hour + 0.5 * (ez_rideshares['Flight depart'].dt.minute//30)

In [9]:
ampl = AMPL()
ampl.eval(r"""
    set A;
    set S;
    set P = 0.25..24 by 0.25;  # time of arrival to airport
    set C;

    param cost{A, C};
    param capacity{C};
    param travelTime{A};
    param flightTime{S, A};

    var numCars{A, P, C} >= 0 integer;  # x
    var leaving{A, S, P} binary;  # 1 if student s leaves for airport a at time p, else 0
    var departureTime{A, S};

    minimize objective: sum{a in A, p in P, c in C} numCars[a, p, c]*cost[a, c];

    subject to notTooLateConstraint{a in A, s in S, p in P}:
        p - leaving[a, s, p]*(flightTime[s, a] - 2 - travelTime[a]) <= 100*(1 - leaving[a, s, p]);

    subject to notTooEarlyConstraint{a in A, s in S, p in P}:
        leaving[a, s, p]*(flightTime[s, a] - 4 - travelTime[a]) - p <= 100*(1 - leaving[a, s, p]);

    subject to capacityConstraint{a in A, p in P}:
        sum{s in S} leaving[a, s, p] <= sum{c in C} numCars[a, p, c] * capacity[c];

    subject to onlyOneActiveY{s in S}:
        sum{a in A, p in P} leaving[a, s, p] = 1;

    subject to defineDepartureTime{a in A, s in S}:
        departureTime[a, s] = sum{p in P} leaving[a, s, p] * p;

""")

In [10]:
# reset index on ez_rideshares to start at 0
ez_rideshares.reset_index(drop=True, inplace=True)

In [11]:
rideshares

,Name,Airport,Departure Date,Flight depart,Earliest Airport Arrival Time,Latest Airport Arrival Time,Flight times
0,Nithya Yeluri,ONT,11/22/2022,1900-01-01 07:00:00,5:30,5:30,7.0
1,Nina Jobanputra,ONT,11/22/2022,1900-01-01 12:15:00,11:00,11:15,12.0
2,Aanya Pratapneni,ONT,11/22/2022,1900-01-01 13:25:00,11:00,11:30,13.0
3,Will Sedo,ONT,11/22/2022,1900-01-01 13:29:00,11:30,12:30,13.0
4,Marcella Todd,ONT,11/22/2022,1900-01-01 15:00:00,11:00,13:00,15.0
5,Kaanthi Pandhigunta,ONT,11/22/2022,1900-01-01 14:45:00,12:00,13:45,14.5
6,Cevi Bainton,ONT,11/22/2022,1900-01-01 15:50:00,13:00,15:00,15.5
7,Ally Dye,ONT,11/22/2022,1900-01-01 16:15:00,14:00,15:00,16.0
8,Maya Maranto,ONT,11/22/2022,1900-01-01 18:05:00,16:30,17:15,18.0
9,Alec Vercruysse,ONT,11/22/2022,1900-01-01 17:55:00,16:30,17:10,17.5


In [12]:
# set flight time data equal to Flight Times form ez_rideshares
# format: {(ez_rideshares['Name'][i], 'ONT') : ez_rideshares['Flight times'][i]}

raw_flight_time_data = {}
for i in range(len(ez_rideshares)):
    raw_flight_time_data[(ez_rideshares['Name'][i], ez_rideshares['Airport'][i])] = ez_rideshares['Flight times'][i]

In [13]:
flight_time_data = {}
for s in ez_rideshares['Name']:
    for j in airports:
        if (s, j) in raw_flight_time_data:
            flight_time_data[(s, j)] = raw_flight_time_data[(s, j)]
        else:
            flight_time_data[(s, j)] = 0

In [14]:
# read ez_rideshares into ampl2
ampl.set['A'] = airports
ampl.set['S'] = ez_rideshares['Name']
ampl.set['C'] = car_types

ampl.getParameter("cost").setValues(cost_data)
ampl.getParameter("capacity").setValues(capacity_data)
ampl.getParameter("travelTime").setValues(travel_time_data)
ampl.getParameter("flightTime").setValues(flight_time_data)


In [15]:
# run the model
ampl.solve(solver='gurobi')

Gurobi 11.0.3:Gurobi 11.0.3: optimal solution; objective 550
231 simplex iterations
14 branching nodes


In [16]:
# display the results
student_airport_times = {}
for student in ez_rideshares['Name']:
    for a in airports:
        x = ampl.getValue(f'departureTime["{a}", "{student}"]')
        if x != 0:
            student_airport_times[student] = (a, x)
student_airport_times

{'Nithya Yeluri': ('ONT', 2.5),
 'Nina Jobanputra': ('ONT', 9.5),
 'Aanya Pratapneni ': ('ONT', 9.5),
 'Will Sedo': ('ONT', 9.5),
 'Marcella Todd': ('ONT', 11.75),
 'Kaanthi Pandhigunta': ('ONT', 11.75),
 'Cevi Bainton': ('ONT', 11.75),
 'Ally Dye': ('ONT', 11.75),
 'Maya Maranto': ('ONT', 15),
 'Alec Vercruysse': ('ONT', 15),
 'Rebecca Preis': ('ONT', 15),
 'Susan Li': ('ONT', 15),
 'Tanvi Krishnan ': ('ONT', 16.5),
 'Jennifer Li': ('ONT', 16.5),
 'Allison Marten': ('ONT', 16.5),
 'Aldrin Feliciano': ('ONT', 16.5),
 'Julianna Schalkwyk': ('ONT', 19),
 'Charles Weismann': ('ONT', 2.5),
 'Katrina Nguyen': ('ONT', 2.5),
 'Bennet Matazzoni': ('ONT', 11.75),
 'Thaxter Shaw': ('LAX', 17),
 'Alicia Krasner': ('LAX', 7.5),
 'Laura Vairus': ('LAX', 12),
 'Ashley Kim': ('SNA', 15.5)}

In [17]:
for i in range(1, 49):
    for a in airports:
        for car in car_types:
            numCars = ampl.getValue(f"numCars['{a}', {i/2}, '{car}']")
            if numCars > 0:
                print(f'Time {i/2} has {numCars} {car} car(s) to {a}')

Time 2.5 has 1 UberX car(s) to ONT
Time 7.5 has 1 UberX car(s) to LAX
Time 9.5 has 1 UberX car(s) to ONT
Time 12.0 has 1 UberX car(s) to LAX
Time 15.0 has 1 UberX car(s) to ONT
Time 15.5 has 1 UberX car(s) to SNA
Time 16.5 has 1 UberX car(s) to ONT
Time 17.0 has 1 UberX car(s) to LAX
Time 19.0 has 1 UberX car(s) to ONT
